In [1]:
import glob
import numpy as np
import pandas as pd 

In [2]:
glob.glob('Garbage classification/*')

['Garbage classification\\cardboard',
 'Garbage classification\\glass',
 'Garbage classification\\metal',
 'Garbage classification\\paper',
 'Garbage classification\\plastic',
 'Garbage classification\\trash']

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
def prepare_dataset(path,label):
    x_train=[]
    y_train=[]
    all_images_path=glob.glob(path+'/*.jpg')
    for img_path in all_images_path :
            img=load_img(img_path, target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            x_train.append(img)
            y_train.append(label)
    return np.array(x_train),np.array(y_train)

In [5]:
paths=glob.glob('Garbage classification/*')
l=len('Garbage classification/')
labels=[]
for path in paths:
    labels.append(path[l:])
    print(labels)

['cardboard']
['cardboard', 'glass']
['cardboard', 'glass', 'metal']
['cardboard', 'glass', 'metal', 'paper']
['cardboard', 'glass', 'metal', 'paper', 'plastic']
['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [33]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

dataset_home = r'Garbage_classification/'
subdirs = [r'train/', r'test/']
for subdir in subdirs:
    # create label subdirectories
    labeldirs = labels
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        makedirs(newdir, exist_ok=True)
        
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = r'Garbage_classification'
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = r'train/'
    if random() < val_ratio:
        dst_dir = r'test/'
    if file.startswith('cardboard'):
        dst = dataset_home + dst_dir + 'cardboard/'  + file
        copyfile(src, dst)
    elif file.startswith('glass'):
        dst = dataset_home + dst_dir + 'glass/'  + file
        copyfile(src, dst)
    elif file.startswith('metal'):
        dst = dataset_home + dst_dir + 'metal/'  + file
        copyfile(src, dst)
    elif file.startswith('paper'):
        dst = dataset_home + dst_dir + 'paper/'  + file
        copyfile(src, dst)
    elif file.startswith('plastic'):
        dst = dataset_home + dst_dir + 'plastic/'  + file
        copyfile(src, dst)
    elif file.startswith('trash'):
        dst = dataset_home + dst_dir + 'trash/'  + file
        copyfile(src, dst)

PermissionError: [Errno 13] Permission denied: 'Garbage_classification/cardboard'

In [6]:
trainX_cardboard, trainY_cardboard  = prepare_dataset("Garbage classification/cardboard",0)
trainX_glass,trainY_glass  = prepare_dataset("Garbage classification/glass",1)
trainX_metal,trainY_metal  = prepare_dataset("Garbage classification/metal",2)
trainX_paper,trainY_paper  = prepare_dataset("Garbage classification/paper",3)
trainX_plastic,trainY_plastic  = prepare_dataset("Garbage classification/plastic",4)
trainX_trash,trainY_trash  = prepare_dataset("Garbage classification/trash",5)

print('train cardboard shape ', trainX_cardboard.shape, trainY_cardboard.shape) 
print('train glass shape', trainX_glass.shape ,trainY_glass.shape)
print('train metal shape', trainX_metal.shape,trainY_metal.shape)
print('train paper shape', trainX_paper.shape, trainY_paper.shape)
print('train plastic shape', trainX_plastic.shape, trainY_plastic.shape)
print('train trash shape', trainX_trash.shape ,trainY_trash.shape)

train cardboard shape  (403, 150, 150, 3) (403,)
train glass (501, 150, 150, 3) (501,)
train metal (410, 150, 150, 3) (410,)
train paper (594, 150, 150, 3) (594,)
train plastic (482, 150, 150, 3) (482,)
train trash (137, 150, 150, 3) (137,)


In [7]:
x_train=np.concatenate((trainX_cardboard,trainX_glass,trainX_metal,trainX_paper,trainX_plastic,trainX_trash),axis=0)
y_train=np.concatenate((trainY_cardboard,trainY_glass,trainY_metal,trainY_paper,trainY_plastic,trainY_trash),axis=0)


In [8]:
print(x_train.shape)
print(y_train.shape)

(2527, 150, 150, 3)
(2527,)


In [ ]:
testX_cardboard, testY_cardboard  = prepare_dataset("Garbage classification/cardboard",0)
testX_glass,testY_glass  = prepare_dataset("Garbage classification/glass",1)
testX_metal,testY_metal  = prepare_dataset("Garbage classification/metal",2)
testX_paper,testY_paper  = prepare_dataset("Garbage classification/paper",3)
testX_plastic,testY_plastic  = prepare_dataset("Garbage classification/plastic",4)
testX_trash,testY_trash  = prepare_dataset("Garbage classification/trash",5)

x_test=np.concatenate((testX_cardboard,testX_glass,testX_metal,testX_paper,testX_plastic,testX_trash),axis=0)
y_test=np.concatenate((testY_cardboard,testY_glass,testY_metal,testY_paper,testY_plastic,testY_trash),axis=0)

In [ ]:
from tensorflow.keras.applications import VGG16

pretrained_model=VGG16(input_shape = (150, 150, 3), 
                        include_top = False, 
                        weights = 'imagenet')

for layer in pretrained_model.layers:
     layer.trainable = False

# pretrained_model.summary()
last_layer = pretrained_model.get_layer('block5_pool')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output= last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)           

model_vgg = Model(pretrained_model.input, x) 


model_vgg.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

model_vgg.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))